In [1]:
pip install pandas numpy scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\lzyda\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
## Data Acquisition and Processing 

In [5]:
import pandas as pd

nhgh_file_path = "./nhgh.tsv"
data = pd.read_csv(nhgh_file_path, sep='\t')

data.head()

,seqn,sex,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,51624,male,34.166667,Non-Hispanic White,"[25000,35000)",0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,51626,male,16.833333,Non-Hispanic Black,"[45000,55000)",0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,51628,female,60.166667,Non-Hispanic Black,"[10000,15000)",1,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,51629,male,26.083333,Mexican American,"[25000,35000)",0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,51630,female,49.666667,Non-Hispanic White,"[35000,45000)",0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79


In [15]:
# Convert to tidy format and ensure data types are correct and consistent

# Check for missing values
missing_values = data.isnull().sum()

# Normalize strings to lowercase
data.columns = [col.lower() for col in data.columns]

print("Missing Values:\n", missing_values)
print("\nData Types:\n", data.dtypes)

Missing Values:
 seqn         0
sex          0
age          0
re           0
income     320
tx           0
dx           0
wt           0
ht           0
bmi          0
leg        231
arml       179
armc       188
waist      239
tri        481
sub        971
gh           0
albumin     89
bun         89
scr         89
dtype: int64

Data Types:
 seqn         int64
sex         object
age        float64
re          object
income      object
tx           int64
dx           int64
wt         float64
ht         float64
bmi        float64
leg        float64
arml       float64
armc       float64
waist      float64
tri        float64
sub        float64
gh         float64
albumin    float64
bun          Int64
scr        float64
dtype: object


In [13]:
# Convert to correct type based on data repo
# Convert 'bun' to numeric, coerce errors, and then to int
data['bun'] = pd.to_numeric(data['bun'], errors='coerce').astype('Int64')

In [16]:
# Double check if string type enums follows the legend
sex_types = ["male", "female"]

re_types = ["mexican american", "other hispanic", "non-hispanic white",
           "non-hispanic black", "other race including multi-racial"]

income_types = ["[0,5000)", "[5000,10000)", "[10000,15000)", "[15000,20000)",
                "[20000,25000)", "[25000,35000)", "[35000,45000)", 
                "[45000,55000)", "[55000,65000)", "[65000,75000)",
                "> 20000", "< 20000", "[75000,100000)", ">= 100000"]

# Check for inconsistencies in 'sex'
sex_check = data['sex'].str.lower().isin(sex_types)

# Check for inconsistencies in 're'
re_check = data['re'].str.lower().isin(re_types)

# Check for inconsistencies in 'income'
income_check = data['income'].str.lower().isin(income_types)

# Count of rows not matching the types
non_matching_sex = data[~sex_check].shape[0]
non_matching_re = data[~re_check].shape[0]
non_matching_income = data[data['income'].notna() \ # there are 320 na values
                           & ~income_check].shape[0]


(non_matching_sex, non_matching_re, non_matching_income)
# Ideally, we want (0,0,0), which indicates all values follow the the enum values

(0, 0, 0)

In [20]:
# SQLite3 Integration and export
import sqlite3

with sqlite3.connect("./diabetes_data.db") as conn:
    data.to_sql("diabetes_data", conn, if_exists="replace", index=False)

In [24]:
# Testing some SQL Queries:

# get number of people with diabetes
with sqlite3.connect('./diabetes_data.db') as conn:
    query = """
    SELECT COUNT(*)
    FROM diabetes_data
    WHERE gh >= 6.5
    """
    female_over_50 = pd.read_sql_query(query, conn)

# Display the first few rows of the retrieved data
female_over_50.head()


,COUNT(*)
0,628
